# 1 - Setup

## 1.1 - Imports & configs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
pd.set_option('display.max_columns', None)

## 1.2 - Funções de ajuda

In [2]:
def checkNaN(x):
    # Recebe um DataFrame e retorna a quantidade de valores nulos e não nulos.
    null_count = x.isna().sum()
    non_null_count = x.notnull().sum()
    print(f"Null values:\n {null_count}")
    print(f"Non-null values:\n {non_null_count}")
    return

def checkOutliers(x):
    """Recebe um DataFrame e retorna um DataFrame com os outliers.
    Baseia-se no pressuposto de que os dados estão normalmente distribuídos."""
    outliers_list = []
    # Itera sobre as colunas numéricas
    for column in x.select_dtypes(include=[np.number]).columns:
        # Calcula o 1 quartil (Q1)
        Q1 = x[column].quantile(0.25)
        # Calcula o 3 quartil (Q3)
        Q3 = x[column].quantile(0.75)
        # Calcula o Intervalo Interquartil (IQR)
        IQR = Q3 - Q1
        # Define os limites inferior e superior para outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Identifica os outliers da coluna atual e adiciona à lista
        outliers = x[(x[column] < lower_bound) | (x[column] > upper_bound)]
        outliers_list.append(outliers)
    # Concatena todos os outliers identificados
    dfOutliers = pd.concat(outliers_list).drop_duplicates().reset_index(drop=True)
    return dfOutliers

def corr(x):
    plt.figure(figsize=(10,8))
    sns.heatmap(x.select_dtypes(include=[np.number]).corr(), annot=True)
    plt.show()
    return

def hist(x):
    # Recebe um DataFrame e plota um histograma para cada coluna numérica.
    x.hist(bins=50, figsize=(25,10))
    plt.show()
    return

def extract_words(column_name):
    # Encontra todas as palavras dentro das aspas simples
    matches = re.findall(r'\'(.*?)\'', column_name)
    # Remove espaços extras e junta as palavras encontradas
    return ' '.join(match.strip() for match in matches)

## 1.3 - Carregamento de Dados

In [32]:
DF_RAW = pd.read_csv("./dataset/data.csv")

# 2 - Análise Inicial

Há muitos NaNs, pois algumas perguntas são exclusivas, por exemplo, se o entrevistado respondeu a pergunta se estava trabalhando com "não", as próximas perguntas que seriam sobre o trabalho foram puladas.<br>
Além de nomes de colunas muito "complexos", há muito texto em certas celulas e algumas celulas redundantes, como por exemplo: uma célula para guardar em formato de texto as tecnologias que são usadas no trabalho do entrevistado, e logo em seguida células com a mesma função, porem com valores booleanos. Isto dificulta um pouco a clara vizualização dos dados.<br>
Por uma coluna ou outra, ~93% das linhas cairam como outliers, talvez valha a pena investigar mais a fundo.<br>
Pelo grande número de colunas e seus nomes pouco intuitivos, fica difícil fazer a matriz de correlação e um histograma geral por hora, então será deixada para a próxima etapa.

In [4]:
DF_RAW.sample(10)

,"('P0', 'id')","('P1_a ', 'Idade')","('P1_a_1 ', 'Faixa idade')","('P1_b ', 'Genero')","('P1_c ', 'Cor/raca/etnia')","('P1_d ', 'PCD')","('P1_e ', 'experiencia_profissional_prejudicada')","('P1_e_1 ', 'Não acredito que minha experiência profissional seja afetada')","('P1_e_2 ', 'Experiencia prejudicada devido a minha Cor Raça Etnia')","('P1_e_3 ', 'Experiencia prejudicada devido a minha identidade de gênero')",...,"('P8_d_3 ', 'Entrando em contato com os times de negócio para definição do problema, identificar a solução e apresentação de resultados.')","('P8_d_4 ', 'Desenvolvendo modelos de Machine Learning com o objetivo de colocar em produção em sistemas (produtos de dados).')","('P8_d_5 ', 'Colocando modelos em produção, criando os pipelines de dados, APIs de consumo e monitoramento.')","('P8_d_6 ', 'Cuidando da manutenção de modelos de Machine Learning já em produção, atuando no monitoramento, ajustes e refatoração quando necessário.')","('P8_d_7 ', 'Realizando construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik, etc.')","('P8_d_8 ', 'Utilizando ferramentas avançadas de estatística como SAS, SPSS, Stata etc, para realizar análises.')","('P8_d_9 ', 'Criando e dando manutenção em ETLs, DAGs e automações de pipelines de dados.')","('P8_d_10 ', 'Criando e gerenciando soluções de Feature Store e cultura de MLOps.')","('P8_d_11 ', 'Criando e mantendo a infra que meus modelos e soluções rodam (clusters, servidores, API, containers, etc.)')","('P8_d_12 ', 'Treinando e aplicando LLM's para solucionar problemas de negócio.')"
3444,nqwst1yddd2th30s6i7ynqwst7rya2i7,50,50-54,Masculino,Preta,Não,"Sim, acredito que a minha a experiência profis...",0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2739,inxf8fcn5tkoblx8o7inxfize19wvj93,30,30-34,Feminino,Branca,Sim,"Sim, acredito que a minha a experiência profis...",0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4328,tm7g55r3x3xowh1itm7gj2kwfjjzhb3q,34,30-34,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4621,vgmj0ipj0hbu9fvglgkevgmj0886cyci,32,30-34,Masculino,Amarela,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2530,hczxujnu5v310qsguqm2nhczxujguorf,27,25-29,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2963,kep9x1igyfczutd6h6wkkep9ascgu47a,21,17-21,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1260,8sfoprtv3nkyd8fq6m8sfvamkcqzo127,19,17-21,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4421,u7sfzryc93bjpxu7sfg3g1qdlidknwv8,51,50-54,Masculino,Parda,Não,"Sim, acredito que a minha a experiência profis...",0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4596,vbhc7vntw3nmoyw0ngk3e1ivbhc7vm3t,37,35-39,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2877,jt8vkuh7ki7wg8qb5prwjt8vku0vmn5x,46,45-49,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
DF_RAW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5293 entries, 0 to 5292
Columns: 399 entries, ('P0', 'id') to ('P8_d_12 ', 'Treinando e aplicando LLM's para solucionar problemas de negócio.')
dtypes: float64(328), int64(2), object(69)
memory usage: 16.1+ MB


In [6]:
checkNaN(DF_RAW)

Null values:
 ('P0', 'id')                                                                                                                     0
('P1_a ', 'Idade')                                                                                                               0
('P1_a_1 ', 'Faixa idade')                                                                                                       0
('P1_b ', 'Genero')                                                                                                              0
('P1_c ', 'Cor/raca/etnia')                                                                                                      0
                                                                                                                              ... 
('P8_d_8 ', 'Utilizando ferramentas avançadas de estatística como SAS, SPSS, Stata etc, para realizar análises.')             4545
('P8_d_9 ', 'Criando e dando manutenção em ETLs, DAGs e automações de

In [7]:
checkOutliers(DF_RAW)

,"('P0', 'id')","('P1_a ', 'Idade')","('P1_a_1 ', 'Faixa idade')","('P1_b ', 'Genero')","('P1_c ', 'Cor/raca/etnia')","('P1_d ', 'PCD')","('P1_e ', 'experiencia_profissional_prejudicada')","('P1_e_1 ', 'Não acredito que minha experiência profissional seja afetada')","('P1_e_2 ', 'Experiencia prejudicada devido a minha Cor Raça Etnia')","('P1_e_3 ', 'Experiencia prejudicada devido a minha identidade de gênero')",...,"('P8_d_3 ', 'Entrando em contato com os times de negócio para definição do problema, identificar a solução e apresentação de resultados.')","('P8_d_4 ', 'Desenvolvendo modelos de Machine Learning com o objetivo de colocar em produção em sistemas (produtos de dados).')","('P8_d_5 ', 'Colocando modelos em produção, criando os pipelines de dados, APIs de consumo e monitoramento.')","('P8_d_6 ', 'Cuidando da manutenção de modelos de Machine Learning já em produção, atuando no monitoramento, ajustes e refatoração quando necessário.')","('P8_d_7 ', 'Realizando construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik, etc.')","('P8_d_8 ', 'Utilizando ferramentas avançadas de estatística como SAS, SPSS, Stata etc, para realizar análises.')","('P8_d_9 ', 'Criando e dando manutenção em ETLs, DAGs e automações de pipelines de dados.')","('P8_d_10 ', 'Criando e gerenciando soluções de Feature Store e cultura de MLOps.')","('P8_d_11 ', 'Criando e mantendo a infra que meus modelos e soluções rodam (clusters, servidores, API, containers, etc.)')","('P8_d_12 ', 'Treinando e aplicando LLM's para solucionar problemas de negócio.')"
0,083f331tparwm4083fxbusb0cps0rmwa,53,50-54,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0baz3khrxi0sei0bazgztvrrugudi1wd,50,50-54,Feminino,Parda,Não,"Sim, acredito que a minha a experiência profis...",0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0es5j6zi3eta53c3alanv0es5j6zm7bp,62,55+,Masculino,Parda,Não,"Sim, acredito que a minha a experiência profis...",0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0sb17993f17dly32q0sb17m6cgr8m8fg,50,50-54,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0t33h6p5dhvgrz7e2vr0t33h6pxixdye,50,50-54,Feminino,Parda,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4904,4fjflyfcspk6owc5dh4fjagf8cfxyqsg,31,30-34,Feminino,Branca,Não,"Sim, acredito que a minha a experiência profis...",0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4905,4bm6svcdx1qpy5pofu4bm6svebowo6g1,33,30-34,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4906,xnwrx1ajipykvjai3xnwrjomf9bwh8wp,25,25-29,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4907,jukw0u6aye00wo0ihckmqbhjukw0uzas,37,35-39,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
DF_RAW.describe()

,"('P1_a ', 'Idade')","('P1_e_1 ', 'Não acredito que minha experiência profissional seja afetada')","('P1_e_2 ', 'Experiencia prejudicada devido a minha Cor Raça Etnia')","('P1_e_3 ', 'Experiencia prejudicada devido a minha identidade de gênero')","('P1_e_4 ', 'Experiencia prejudicada devido ao fato de ser PCD')","('P1_f_1', 'Quantidade de oportunidades de emprego/vagas recebidas')","('P1_f_2', 'Senioridade das vagas recebidas em relação à sua experiência')","('P1_f_3', 'Aprovação em processos seletivos/entrevistas')","('P1_f_4', 'Oportunidades de progressão de carreira')","('P1_f_5', 'Velocidade de progressão de carreira')",...,"('P8_d_3 ', 'Entrando em contato com os times de negócio para definição do problema, identificar a solução e apresentação de resultados.')","('P8_d_4 ', 'Desenvolvendo modelos de Machine Learning com o objetivo de colocar em produção em sistemas (produtos de dados).')","('P8_d_5 ', 'Colocando modelos em produção, criando os pipelines de dados, APIs de consumo e monitoramento.')","('P8_d_6 ', 'Cuidando da manutenção de modelos de Machine Learning já em produção, atuando no monitoramento, ajustes e refatoração quando necessário.')","('P8_d_7 ', 'Realizando construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik, etc.')","('P8_d_8 ', 'Utilizando ferramentas avançadas de estatística como SAS, SPSS, Stata etc, para realizar análises.')","('P8_d_9 ', 'Criando e dando manutenção em ETLs, DAGs e automações de pipelines de dados.')","('P8_d_10 ', 'Criando e gerenciando soluções de Feature Store e cultura de MLOps.')","('P8_d_11 ', 'Criando e mantendo a infra que meus modelos e soluções rodam (clusters, servidores, API, containers, etc.)')","('P8_d_12 ', 'Treinando e aplicando LLM's para solucionar problemas de negócio.')"
count,5293.000000,2719.000000,2719.000000,2719.000000,2719.000000,1245.000000,1245.000000,1245.000000,1245.000000,1245.000000,...,748.000000,748.000000,748.000000,748.000000,748.000000,748.000000,748.000000,748.000000,748.000000,748.000000
mean,31.997166,0.542111,0.202648,0.291284,0.024641,0.389558,0.342972,0.413655,0.490763,0.516466,...,0.243316,0.315508,0.070856,0.093583,0.045455,0.009358,0.025401,0.020053,0.024064,0.048128
std,7.623498,0.498315,0.402046,0.454437,0.155058,0.487846,0.474893,0.492686,0.500116,0.499930,...,0.429371,0.465029,0.256755,0.291442,0.208438,0.096349,0.157445,0.140277,0.153351,0.214181
min,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,30.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,36.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,73.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# 3 - Limpeza e Transformação

In [9]:
df_clean = DF_RAW.copy()

## 3.1 - Limpeza de strings

In [10]:
# Concertar typo em nome de coluna
df_clean = df_clean.rename(columns={"('P6_b_16 ', 'SQL Server Integration Services (SSIS))": "('P6_b_16 ', 'SQL Server Integration Services (SSIS)')"})
# Remover excesso de caracteres do nome das colunas
df_clean.columns = df_clean.columns.map(extract_words)
# Remover espaços em branco em inicios e fims de valores strings, se houver
df_clean = df_clean.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# Limpar texto na coluna "Regiao de origem"
df_clean["P1_k Regiao de origem"] = df_clean["P1_k Regiao de origem"].str.extract(r'\((.*?)\)', expand=False)

## 3.2 - Remover colunas redundantes e irrelevantes para a análise

In [11]:
# Remover colunas redundantes
df_clean = df_clean.drop(columns=[
    "P1_a_1 Faixa idade", "P1_e experiencia_profissional_prejudicada", "P1_f aspectos_prejudicados", "P1_i Estado onde mora", "P1_i_2 Regiao onde mora",
    "P2_l Qual o principal motivo da sua insatisfação com a empresa atual?", "P2_o Quais os principais critérios que você leva em consideração no momento de decidir onde trabalhar?",
    "P3_b Quais desses papéis/cargos fazem parte do time (ou chapter) de dados da sua empresa?", "P3_c Quais dessas responsabilidades fazem parte da sua rotina atual de trabalho como gestor?",
    "P3_d Quais são os 3 maiores desafios que você tem como gestor no atual momento?", "P3_f Tipos de uso de AI Generativa e LLMs na empresa", "P3_f_8 Não sei opinar sobre o uso de IA Generativa e LLMs na empresa",
    "P3_g Motivos que levam a empresa a não usar AI Genrativa e LLMs", "P4_b Quais das fontes de dados listadas você já analisou ou processou no trabalho?",
    "P4_c Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?", "P4_d Quais das linguagens listadas abaixo você utiliza no trabalho?", "P4_d_15 Não utilizo nenhuma linguagem",
    "P4_g Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?", "P4_h Dentre as opções listadas, qual sua Cloud preferida?", "P4_j Ferramenta de BI utilizada no dia a dia",
    "P4_j_23 Não utilizo nenhuma ferramenta de BI no trabalho", "P4_l Qual o tipo de uso de AI Generativa e LLMs na empresa", "P4_l_8 Não sei opinar sobre o uso de IA Generativa e LLMs na empresa",
    "P4_m Utiliza ChatGPT ou LLMs no trabalho?", "P6_a Quais das opções abaixo fazem parte da sua rotina no trabalho atual como engenheiro de dados?", "P6_a_9 Nenhuma das opções listadas refletem meu dia a dia.",
    "P6_b Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Engineer?", "P6_b_21 Não utilizo ferramentas de ETL", "P6_d Qual tecnologia utilizada como plataforma do Data Lake?",
    "P6_f Qual tecnologia utilizada como plataforma do Data Warehouse?", "P6_h Em qual das opções abaixo você gasta a maior parte do seu tempo?", "P6_h_9 Nenhuma das opções listadas refletem meu dia a dia.",
    "P7_1 Quais das opções abaixo fazem parte da sua rotina no trabalho atual com análise de dados?", "P7_b Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Analyst?",
    "P7_b_21 Não utilizo ferramentas de ETL", "P7_c Sua empresa utiliza alguma das ferramentas listadas para dar mais autonomia em análise de dados para as áreas de negócio?",
    "P7_c_5 Minha empresa não utiliza essas ferramentas.", "P7_c_6 Não sei informar.", "P7_d Em qual das opções abaixo você gasta a maior parte do seu tempo de trabalho?", "P7_d_10 Nenhuma das opções listadas refletem meu dia a dia.", "P8_a Quais das opções abaixo fazem parte da sua rotina no trabalho atual com ciência de dados?", "P8_b Quais as técnicas e métodos listados abaixo você costuma utilizar no trabalho?", "P8_3 Quais dessas tecnologias fazem parte do seu dia a dia como cientista de dados?", "P8_d Em qual das opções abaixo você gasta a maior parte do seu tempo no trabalho?", "P1_e_1 Não acredito que minha experiência profissional seja afetada",
    "P3_c_1 Pensar na visão de longo prazo de dados da empresa e fortalecimento da cultura analítica da companhia.", "P3_c_2 Organização de treinamentos e iniciativas com o objetivo de aumentar a maturidade analítica das áreas de negócios.", "P3_c_3 Atração, seleção e contratação de talentos para o time de dados.", "P3_c_4 Decisão sobre contratação de ferramentas e tecnologias relacionadas a dados.",
    "P3_c_5 Sou gestor da equipe responsável pela engenharia de dados e por manter o Data Lake da empresa como fonte única dos dados, garantindo a qualidade e confiabilidade da informação.",
    "P3_c_6 Sou gestor da equipe responsável pela entrega de dados, estudos, relatórios e dashboards para as áreas de negócio da empresa.",
    "P3_c_7 Sou gestor da equipe responsável por iniciativas e projetos envolvendo Inteligência Artificial e Machine Learning.",
    "P3_c_9 Gestão de projetos de dados, cuidando das etapas, equipes envolvidas, atingimento dos objetivos etc.",
    "P3_c_10 Gestão de produtos de dados, cuidando da visão dos produtos, backlog, feedback de usuários etc.", "P3_c_11 Gestão de pessoas, apoio no desenvolvimento das pessoas, evolução de carreira",
    "P3_d_4 d Gestão de equipes no ambiente remoto.", "P3_d_5 e Gestão de projetos envolvendo áreas multidisciplinares da empresa.", "P3_d_6 f Organizar as informações e garantir a qualidade e confiabilidade.",
    "P3_d_7 g Conseguir processar e armazenar um alto volume de dados.", "P3_d_8 h Conseguir gerar valor para as áreas de negócios através de estudos e experimentos.",
    "P3_d_9 i Desenvolver e manter modelos Machine Learning em produção.", "P3_d_10 j Gerenciar a expectativa das áreas de negócio em relação as entregas das equipes de dados.",
    "P3_d_11 k Garantir a manutenção dos projetos e modelos em produção, em meio ao crescimento da empresa.", "P3_d_14 Dividir o tempo entre entregas técnicas e gestão.", "P3_f_7 IA Generativa e LLMs não é prioridade",
    "P3_g_1 Falta de compreensão dos casos de uso",	"P3_g_2 Falta de confiabilidade das saídas (alucinação dos modelos)", "P3_g_3 Incerteza em relação a regulamentação",
    "P3_g_4 Preocupações com segurança e privacidade de dados",	"P3_g_5 Retorno sobre investimento (ROI) não comprovado de IA Generativa",	"P3_g_6 Dados da empresa não estão prontos para uso de IA Generativa",
    "P3_g_7 Falta de expertise ou falta de recursos", "P3_g_8 Alta direção da empresa não vê valor ou não vê como prioridade", "P3_g_9 Preocupações com propriedade intelectual",
    "P4_b_1 Dados relacionais (estruturados em bancos SQL)", "P4_b_2 Dados armazenados em bancos NoSQL", "P4_b_3 Imagens", "P4_b_4 Textos/Documentos" ,"P4_b_5 Vídeos",	"P4_b_6 Áudios", "P4_b_7 Planilhas",
    "P4_b_8 Dados georeferenciados", "P4_c_1 Dados relacionais (estruturados em bancos SQL)", "P4_c_2 Dados armazenados em bancos NoSQL", "P4_c_3 Imagens", "P4_c_4 Textos/Documentos", "P4_c_5 Vídeos", "P4_c_6 Áudios",
    "P4_c_7 Planilhas",	"P4_c_8 Dados georeferenciados", "P3_b_1 Analytics Engineer", "P3_b_2 Engenharia de Dados/Data Engineer", "P3_b_3 Analista de Dados/Data Analyst", "P3_b_4 Cientista de Dados/Data Scientist", "P3_b_5 Database Administrator/DBA", "P3_b_6 Analista de Business Intelligence/BI", "P3_b_7 Arquiteto de Dados/Data Architect", "P3_b_8 Data Product Manager/DPM", "P3_b_9 Business Analyst",	"P4_m_1 Não uso soluções de AI Generativa com foco em produtividade", "P6_b_1 Scripts Python", "P6_b_2 SQL & Stored Procedures", "P6_b_3 Apache Airflow", "P6_b_4 Apache NiFi", "P6_b_5 Luigi", "P6_b_6 AWS Glue", "P6_b_7 Talend", "P6_b_8 Pentaho", "P6_b_9 Alteryx", "P6_b_10 Stitch", "P6_b_11 Fivetran", "P6_b_12 Google Dataflow", "P6_b_13 Oracle Data Integrator", "P6_b_14 IBM DataStage", "P6_b_15 SAP BW ETL", "P6_b_16 SQL Server Integration Services (SSIS)", "P6_b_17 SAS Data Integration", "P6_b_18 Qlik Sense", "P6_b_19 Knime", "P6_b_20 Databricks", "P1_j Mudou de Estado?"
    ])
print(f"A diferença entre o dataset original e o limpo é de {DF_RAW.shape[1] - df_clean.shape[1]} colunas")


A diferença entre o dataset original e o limpo é de 121 colunas


In [31]:
df_clean.sample(5)

,P0 id,P1_a Idade,P1_b Genero,P1_c Cor/raca/etnia,P1_d PCD,P1_e_2 Experiencia prejudicada devido a minha Cor Raça Etnia,P1_e_3 Experiencia prejudicada devido a minha identidade de gênero,P1_e_4 Experiencia prejudicada devido ao fato de ser PCD,P1_f_1 Quantidade de oportunidades de emprego/vagas recebidas,P1_f_2 Senioridade das vagas recebidas em relação à sua experiência,P1_f_3 Aprovação em processos seletivos/entrevistas,P1_f_4 Oportunidades de progressão de carreira,P1_f_5 Velocidade de progressão de carreira,P1_f_6 Nível de cobrança no trabalho/Stress no trabalho,P1_f_7 Atenção dada diante das minhas opiniões e ideias,"P1_f_8 Relação com outros membros da empresa, em momentos de trabalho","P1_f_9 Relação com outros membros da empresa, em momentos de integração e outros momentos fora do trabalho",P1_g vive_no_brasil,P1_i_1 uf onde mora,P1_k Regiao de origem,P1_l Nivel de Ensino,P1_m Área de Formação,P2_a Qual sua situação atual de trabalho?,P2_b Setor,P2_c Numero de Funcionarios,P2_d Gestor?,P2_e Cargo como Gestor,P2_f Cargo Atual,P2_g Nivel,P2_h Faixa salarial,P2_i Quanto tempo de experiência na área de dados você tem?,P2_j Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?,P2_k Você está satisfeito na sua empresa atual?,P2_l_1 Falta de oportunidade de crescimento no emprego atual,P2_l_2 Salário atual não corresponde ao mercado,P2_l_3 Não tenho uma boa relação com meu líder/gestor,P2_l_4 Gostaria de trabalhar em em outra área de atuação,P2_l_5 Gostaria de receber mais benefícios,P2_l_6 O clima de trabalho/ambiente não é bom,P2_l_7 Falta de maturidade analítica na empresa,P2_m Você participou de entrevistas de emprego nos últimos 6 meses?,P2_n Você pretende mudar de emprego nos próximos 6 meses?,P2_o_1 Remuneração/Salário,P2_o_2 Benefícios,P2_o_3 Propósito do trabalho e da empresa,P2_o_4 Flexibilidade de trabalho remoto,P2_o_5 Ambiente e clima de trabalho,P2_o_6 Oportunidade de aprendizado e trabalhar com referências na área,P2_o_7 Plano de carreira e oportunidades de crescimento profissional,P2_o_8 Maturidade da empresa em termos de tecnologia e dados,P2_o_9 Qualidade dos gestores e líderes,P2_o_10 Reputação que a empresa tem no mercado,P2_q Empresa que trabaha passou por layoff em 2023,P2_r Atualmente qual a sua forma de trabalho?,P2_s Qual a forma de trabalho ideal para você?,P2_t Caso sua empresa decida pelo modelo 100% presencial qual será sua atitude?,P3_a Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?,"P3_c_8 Apesar de ser gestor ainda atuo na parte técnica, construindo soluções/análises/modelos etc.",P3_d_1 a Contratar novos talentos.,P3_d_2 b Reter talentos.,P3_d_3 c Convencer a empresa a aumentar os investimentos na área de dados.,P3_d_12 Conseguir levar inovação para a empresa através dos dados.,P3_d_13 Garantir retorno do investimento (ROI) em projetos de dados.,P3_e AI Generativa é uma prioridade em sua empresa?,P3_f_1 Colaboradores usando AI generativa de forma independente e descentralizada,P3_f_2 Direcionamento centralizado do uso de AI generativa,P3_f_3 Desenvolvedores utilizando Copilots,P3_f_4 AI Generativa e LLMs para melhorar produtos externos,P3_f_5 AI Generativa e LLMs para melhorar produtos internos para os colaboradores,P3_f_6 IA Generativa e LLMs como principal frente do negócio,"P4_a Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?",P4_a_1 Atuacao,P4_d_1 SQL,P4_d_2 R,P4_d_3 Python,P4_d_4 C/C++/C#,P4_d_5 .NET,P4_d_6 Java,P4_d_7 Julia,P4_d_8 SAS/Stata,P4_d_9 Visual Basic/VBA,P4_d_10 Scala,P4_d_11 Matlab,P4_d_12 Rust,P4_d_13 PHP,P4_d_14 JavaScript,"P4_e Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?","P4_f Entre as linguagens listadas abaixo, qual é a sua preferida?",P4_g_1 MySQL,P4_g_2 Oracle,P4_g_3 SQL SERVER,P4_g_4 Amazon Aurora ou RDS,P4_g_5 DynamoDB,P4_g_6 CoachDB,P4_g_7 Cassandra,P4_

## 3.x - Transformar tipos de dados

## 3.x - Preencher valores NaNs

## 3.x - Alterar nomes das colunas

## 3.x - Exportar dataset limpo

In [33]:
# Exportar o dataset limpo
df_clean.to_csv('./output/data_clean.csv', index=False)

# 4 - Análise 

## 4.1 - Visão DA

In [ ]:
df_targets = df_clean[df_clean["P2_f Cargo Atual"] == ]

In [30]:
df_clean["P2_f Cargo Atual"].unique()

array(['Cientista de Dados/Data Scientist', 'Analista de BI/BI Analyst',
       'Analista de Dados/Data Analyst', nan,
       'Engenheiro de Dados/Arquiteto de Dados/Data Engineer/Data Architect',
       'Analista de Negócios/Business Analyst', 'Outra Opção',
       'Desenvolvedor/ Engenheiro de Software/ Analista de Sistemas',
       'Professor/Pesquisador', 'Analytics Engineer', 'Economista',
       'Estatístico', 'Analista de Suporte/Analista Técnico',
       'Data Product Manager/ Product Manager (PM/APM/DPM/GPM/PO)',
       'Outras Engenharias (não inclui dev)',
       'Analista de Inteligência de Mercado/Market Intelligence',
       'Engenheiro de Machine Learning/ML Engineer/AI Engineer',
       'DBA/Administrador de Banco de Dados'], dtype=object)

## 4.2 - Visão gestores

In [ ]:
df_gestores = df_clean[df_clean["P2_d Gestor?"] == 1.0]
df_gestores = df_gestores.dropna(axis=1, how='all')
df_gestores.shape

In [28]:
df_heads.sample(5)

,P0 id,P1_a Idade,P1_b Genero,P1_c Cor/raca/etnia,P1_d PCD,P1_e_2 Experiencia prejudicada devido a minha Cor Raça Etnia,P1_e_3 Experiencia prejudicada devido a minha identidade de gênero,P1_e_4 Experiencia prejudicada devido ao fato de ser PCD,P1_f_1 Quantidade de oportunidades de emprego/vagas recebidas,P1_f_2 Senioridade das vagas recebidas em relação à sua experiência,P1_f_3 Aprovação em processos seletivos/entrevistas,P1_f_4 Oportunidades de progressão de carreira,P1_f_5 Velocidade de progressão de carreira,P1_f_6 Nível de cobrança no trabalho/Stress no trabalho,P1_f_7 Atenção dada diante das minhas opiniões e ideias,"P1_f_8 Relação com outros membros da empresa, em momentos de trabalho","P1_f_9 Relação com outros membros da empresa, em momentos de integração e outros momentos fora do trabalho",P1_g vive_no_brasil,P1_i_1 uf onde mora,P1_k Regiao de origem,P1_l Nivel de Ensino,P1_m Área de Formação,P2_a Qual sua situação atual de trabalho?,P2_b Setor,P2_c Numero de Funcionarios,P2_d Gestor?,P2_e Cargo como Gestor,P2_h Faixa salarial,P2_i Quanto tempo de experiência na área de dados você tem?,P2_j Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?,P2_k Você está satisfeito na sua empresa atual?,P2_l_1 Falta de oportunidade de crescimento no emprego atual,P2_l_2 Salário atual não corresponde ao mercado,P2_l_3 Não tenho uma boa relação com meu líder/gestor,P2_l_4 Gostaria de trabalhar em em outra área de atuação,P2_l_5 Gostaria de receber mais benefícios,P2_l_6 O clima de trabalho/ambiente não é bom,P2_l_7 Falta de maturidade analítica na empresa,P2_m Você participou de entrevistas de emprego nos últimos 6 meses?,P2_n Você pretende mudar de emprego nos próximos 6 meses?,P2_o_1 Remuneração/Salário,P2_o_2 Benefícios,P2_o_3 Propósito do trabalho e da empresa,P2_o_4 Flexibilidade de trabalho remoto,P2_o_5 Ambiente e clima de trabalho,P2_o_6 Oportunidade de aprendizado e trabalhar com referências na área,P2_o_7 Plano de carreira e oportunidades de crescimento profissional,P2_o_8 Maturidade da empresa em termos de tecnologia e dados,P2_o_9 Qualidade dos gestores e líderes,P2_o_10 Reputação que a empresa tem no mercado,P2_q Empresa que trabaha passou por layoff em 2023,P2_r Atualmente qual a sua forma de trabalho?,P2_s Qual a forma de trabalho ideal para você?,P2_t Caso sua empresa decida pelo modelo 100% presencial qual será sua atitude?,P3_a Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?,"P3_c_8 Apesar de ser gestor ainda atuo na parte técnica, construindo soluções/análises/modelos etc.",P3_d_1 a Contratar novos talentos.,P3_d_2 b Reter talentos.,P3_d_3 c Convencer a empresa a aumentar os investimentos na área de dados.,P3_d_12 Conseguir levar inovação para a empresa através dos dados.,P3_d_13 Garantir retorno do investimento (ROI) em projetos de dados.,P3_e AI Generativa é uma prioridade em sua empresa?,P3_f_1 Colaboradores usando AI generativa de forma independente e descentralizada,P3_f_2 Direcionamento centralizado do uso de AI generativa,P3_f_3 Desenvolvedores utilizando Copilots,P3_f_4 AI Generativa e LLMs para melhorar produtos externos,P3_f_5 AI Generativa e LLMs para melhorar produtos internos para os colaboradores,P3_f_6 IA Generativa e LLMs como principal frente do negócio,P4_a_1 Atuacao
413,2qmr35cqpg4nvi932qmr35cqlh50eux9,37,Masculino,Branca,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RJ,NaN,Pós-graduação,Estatística/ Matemática / Matemática Computaci...,Empregado (CLT),Varejo,de 1.001 a 3.000,1.0,Gerente/Head,de R$ 20.001/mês a R$ 25.000/mês,Mais de 10 anos,Não tive experiência na área de TI/Engenharia ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Sim, fiz entrevistas mas não fui aprovado (ou ...","Não estou buscando, mas me considero aberto a ...",1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,"Sim, ocorreram layoffs/demissões em massa na e...",Modelo 100% remoto,Modelo 100% remoto,Vou procurar outra oportunidade no modelo 